#### Imports

In [114]:
import os
import re
import skrf as rf
import pandas as pd

#### Paths

In [115]:
script_directory = os.getcwd()#obtem diretório atual
s2ps_path = os.path.join(script_directory, 'oil_data','oil_data') #cria string caminho para todos os .s2p

#### Regex = hell

In [116]:
regex = r'SAW_06_02_2024_OIL_GIRASSOL_(\d+)\.S2P'
#SAW_06_02_2024_OIL_GIRASSOL_01.S2P

In [117]:
def read_s2p_vegetal_data(regex, path):
    dfs = []#lista para ocncatenar todos os dicts de cada medição 
    for filename in os.listdir(path):
        match = re.match(regex, filename)
        if match:
            #obtem informação do nome do arquivo
            oil_name = "Sunflower"
            measurement_number = int(match.group(1))
            
            #carrega arquivo
            path_to_oil = os.path.join(path, filename)
            s2p_file = rf.Network(path_to_oil)
            
            #extrai frequência e parâmetros S do objeto Network
            frequency = s2p_file.f
            s_parameters = s2p_file.s_db
            
            #Dataframe com informações do objeto Network
            individual_oil_dict = {
                'OIL NAME': oil_name,
                'MEASUREMENT NUMBER': measurement_number,
                'FREQUENCY (MHZ)':frequency/1000000,
                'S11 (db)' : s_parameters[:,0,0],
                'S21 (db)' : s_parameters[:,1,0],
                'S12 (db)' : s_parameters[:,0,1],
                'S22 (db)' : s_parameters[:,1,1],
            }
            df = pd.DataFrame.from_dict(individual_oil_dict)
            dfs.append(df)
            
    dataframe = pd.concat(dfs, ignore_index=True)
    return dataframe

#### Read Dataframes

In [118]:
df = read_s2p_vegetal_data(regex, s2ps_path)

In [119]:
df.info

<bound method DataFrame.info of         OIL NAME  MEASUREMENT NUMBER  FREQUENCY (MHZ)  S11 (db)  S21 (db)  \
0      Sunflower                   1       110.000000 -1.019594 -79.25723   
1      Sunflower                   1       110.002000 -1.021273 -85.04324   
2      Sunflower                   1       110.004000 -1.020696 -76.13750   
3      Sunflower                   1       110.006001 -1.018957 -79.30806   
4      Sunflower                   1       110.008001 -1.019897 -81.14824   
...          ...                 ...              ...       ...       ...   
29995  Sunflower                   3       129.991999 -1.383861 -70.31435   
29996  Sunflower                   3       129.993999 -1.384753 -68.17764   
29997  Sunflower                   3       129.996000 -1.386417 -67.99842   
29998  Sunflower                   3       129.998000 -1.384719 -68.36824   
29999  Sunflower                   3       130.000000 -1.384075 -67.22101   

       S12 (db)  S22 (db)  
0     -79.50127

In [120]:
#assegura somente óleo de girassol lido
for name in df["OIL NAME"]:
    if name != "SUNFLOWER":
        print("Not sunflower")

Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not sunflower
Not su

In [121]:
#cria .csv caso precise no futuro.
path_save_csv = os.path.join(script_directory, 'oil_data','oil_data','CSVs')
if not os.path.exists(path_save_csv):
   os.makedirs(path_save_csv)
df.to_csv(path_save_csv + '/sunflower_measurements.csv')

In [122]:
def average_measurements(dataframe):
    #agrupa valores por frequência e calcula média do que interessa
    return dataframe.groupby('FREQUENCY (MHZ)', as_index=False)['S21 (db)'].mean()

In [123]:
df_sunflower_s21 = average_measurements(df)

In [124]:
df_sunflower_s21.info


<bound method DataFrame.info of       FREQUENCY (MHZ)   S21 (db)
0          110.000000 -78.702427
1          110.002000 -80.748823
2          110.004000 -79.251293
3          110.006001 -77.673827
4          110.008001 -79.596690
...               ...        ...
9995       129.991999 -70.084900
9996       129.993999 -68.847677
9997       129.996000 -68.065960
9998       129.998000 -68.482603
9999       130.000000 -67.123187

[10000 rows x 2 columns]>

In [125]:
#cria .csv caso precise no futuro.
path_save_csv_processed = os.path.join(script_directory, 'oil_data','oil_data','CSVs','processed')
if not os.path.exists(path_save_csv_processed):
   os.makedirs(path_save_csv_processed)
df.to_csv(path_save_csv_processed + '/sunflower_measurements_processed.csv')